In [22]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import time
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain.vectorstores import  Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
import pinecone
import os
from dotenv import load_dotenv

In [24]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

MODEL = "text-embedding-3-small"
client = OpenAI(api_key=OPENAI_API_KEY)
spec = ServerlessSpec(cloud="AWS", region="us-east-1")
index_name = 'arpa2'
# print(pc.list_indexes())

In [25]:
pc = Pinecone(api_key=PINECONE_API_KEY)

TypeError: Pinecone.__init__() got an unexpected keyword argument 'api_key'

In [16]:
loader = PyPDFLoader("/Users/luisbarajas/Documents/AGI/Papers/weak-to-strong-generalization.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
print(texts)
# print(len(texts))
# for(text) in texts:
#     print(text.page_content)

# print (f'You have {len(data)} document(s) in your data')
# print (f'There are {len(data[0].page_content)} characters in your document')

[Document(page_content='WEAK-TO-STRONG GENERALIZATION : ELICITING\nSTRONG CAPABILITIES WITHWEAK SUPERVISION\nCollin Burns∗Pavel Izmailov∗Jan Hendrik Kirchner∗Bowen Baker∗Leo Gao∗\nLeopold Aschenbrenner∗Yining Chen∗Adrien Ecoffet∗Manas Joglekar∗\nJan Leike Ilya Sutskever Jeff Wu∗\nOpenAI\nABSTRACT\nWidely used alignment techniques, such as reinforcement learning from human\nfeedback (RLHF), rely on the ability of humans to supervise model behavior—for\nexample, to evaluate whether a model faithfully followed instructions or generated\nsafe outputs. However, future superhuman models will behave in complex ways\ntoo difficult for humans to reliably evaluate; humans will only be able to weakly\nsupervise superhuman models. We study an analogy to this problem: can weak\nmodel supervision elicit the full capabilities of a much stronger model? We test\nthis using a range of pretrained language models in the GPT-4 family on natural\nlanguage processing (NLP), chess, and reward modeling tasks. 

In [17]:
# check if index already exists (it shouldn't if this is your first run)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name=index_name,
        dimension=1536,  # dimensionality of text-embed-3-small
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
        
# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

TypeError: Pinecone.create_index() got an unexpected keyword argument 'index_name'

Modificar el for para iterar sobre el arreglo de obtjetos de tipo documento del archivo

In [21]:
count = 0  # we'll use the count to create unique IDs
batch_size = 32  # process everything in batches of 32
for document in texts:
    text_chunk = document.page_content
    # create embeddings
    embbedings = client.embeddings.create(input=text_chunk, model=MODEL)
    embeds = [record.embedding for record in embbedings.data]

    # prep metadata and upsert batch
    meta = [{'text': text_chunk}]
    to_upsert = zip(count, embeds, meta)
    count += 1
    
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))

  0%|          | 0/32 [00:00<?, ?it/s]

In [22]:
query = "What caused the 1929 Great Depression?"

xq = client.embeddings.create(input=query, model=MODEL).data[0].embedding


In [23]:
res = index.query(vector=xq, top_k=5, include_metadata=True)

In [24]:
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.75: Why did the world enter a global depression in 1929 ?
0.60: When was `` the Great Depression '' ?
0.37: What crop failure caused the Irish Famine ?
0.32: What were popular songs and types of songs in the 1920s ?
0.32: When did World War I start ?
